In [1]:
#%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nibabel as nib
from pathlib import *
from pylabs.utils.paths import getnetworkdataroot
fs = Path(getnetworkdataroot())
pd.set_option('display.width', 999999)
pd.options.display.max_colwidth = 250
pd.options.display.max_rows = 999

In [2]:
vtkf = fs / 'bbc' / 'allvtk_run3.csv'
vtk_fbs = pd.read_csv(vtkf)

In [3]:
#this renames col label 'subjectid' with 'vtk_file_name'
vtk_fbs.rename(columns={'subjectid': 'vtk_file_name', 'icountgood': 'FB_reach_dest', 'icounttotal': 'total_num_FB'}, inplace=True)
#makes subj id fm 1st 10 chars of vtk_file_name
vtk_fbs['Subjid'] = vtk_fbs['vtk_file_name'].str[0:10]
#makes fit methods
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('restore_cam_tensor_medfilt'), 'fit_method'] = 'restore_camino_mf'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('restore_cam_tensor_mori'), 'fit_method'] = 'restore_camino'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('wls_cam_tensor_medfilt'), 'fit_method'] = 'wls_camino_mf'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('wls_cam_tensor_mori'), 'fit_method'] = 'wls_camino'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('ols_cam_tensor_medfilt'), 'fit_method'] = 'ols_camino_mf'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('ols_cam_tensor_mori'), 'fit_method'] = 'ols_camino'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('ols_dipy_tensor_mori'), 'fit_method'] = 'ols_dipy'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('ols_dipy_tensor_med'), 'fit_method'] = 'ols_dipy_mf'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('wls_dipy_tensor_mori'), 'fit_method'] = 'wls_dipy'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('wls_dipy_tensor_medfilt'), 'fit_method'] = 'wls_dipy_mf'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('ols_fsl_tensor_medfilt'), 'fit_method'] = 'ols_fsl_mf'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('ols_fsl_tensor_mori'), 'fit_method'] = 'ols_fsl'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('wls_fsl_tensor_medfilt'), 'fit_method'] = 'wls_fsl_mf'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('wls_fsl_tensor_mori'), 'fit_method'] = 'wls_fsl'
#makes region names for regions col.
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('LeftPostIntCap'), 'region'] = 'LeftPostIntCap'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('RightPostIntCap'), 'region'] = 'RightPostIntCap'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('Left_IFOF-70'), 'region'] = 'Left_IFOF'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('Right_IFOF-158'), 'region'] = 'Right_IFOF'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('mori_CC'), 'region'] = 'Corpus_Callosum'
#calculate %FB arrive at target as float and formatted text for display
vtk_fbs['arrived'] = 1 - vtk_fbs['total_num_FB'].sub(vtk_fbs['FB_reach_dest']).div(vtk_fbs['total_num_FB'])
vtk_fbs['perc_arr'] = pd.Series(["{0:.2f}%".format(val*100) for val in vtk_fbs['arrived']], index = vtk_fbs.index)

In [4]:
#shows flat file no multiIndex
testrange = np.arange(0, len(vtk_fbs), int(len(vtk_fbs)/19))
vtk_fbs.ix[testrange,['Subjid', 'fit_method', 'region', 'vtk_file_name', 'fa', 'md', 'FB_reach_dest', 'total_num_FB', 'perc_arr']]

,Subjid,fit_method,region,vtk_file_name,fa,md,FB_reach_dest,total_num_FB,perc_arr
0,sub-bbc101,restore_camino_mf,Corpus_Callosum,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam_tensor_medfilt_mori_CC.vtk,0.380762,7.872827e-10,2373,2694,88.08%
73,sub-bbc105,restore_camino_mf,Right_IFOF,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam_tensor_medfilt_mori_Right_IFOF-158.vtk,0.372283,7.728811e-10,205,417,49.16%
146,sub-bbc106,ols_fsl,Left_IFOF,sub-bbc106_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_ols_fsl_tensor_mori_Left_IFOF-70.vtk,0.414190,7.839501e-04,41,592,6.93%
219,sub-bbc108,ols_dipy,RightPostIntCap,sub-bbc108_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_ols_dipy_tensor_mori_RightPostIntCap-123.vtk,0.470025,7.608997e-04,134,356,37.64%
292,sub-bbc113,ols_dipy_mf,LeftPostIntCap,sub-bbc113_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_ols_dipy_tensor_medfilt_mori_LeftPostIntCap-35.vtk,0.472423,7.663944e-04,75,330,22.73%
365,sub-bbc116,ols_camino,Corpus_Callosum,sub-bbc116_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_ols_cam_tensor_mori_CC.vtk,0.402936,8.326907e-10,1220,2551,47.82%
438,sub-bbc116,wls_fsl,Right_IFOF,sub-bbc116_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_mori_Right_IFOF-158.vtk,0.386043,6.251742e-04,44,298,14.77%
511,sub-bbc118,wls_fsl_mf,Left_IFOF,sub-bbc118_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_medfilt_mori_Left_IFOF-70.vtk,0.285014,7.830975e-04,41,298,13.76%
584,sub-bbc119,wls_dipy_mf,RightPostIntCap,sub-bbc119_ses-2_dti_15dir_b1000_2_eddy_corrected_repol_std2_wls_dipy_tensor_medfilt_mori_RightPostIntCap-123.vtk,0.410033,9.171873e-04,90,334,26.95%
657,sub-bbc120,wls_camino,LeftPostIntCap,sub-bbc120_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_cam_tensor_mori_LeftPostIntCap-35.vtk,0.453479,7.918401e-10,95,357,26.61%


In [5]:
#make into multiIndex
sid = np.array(vtk_fbs.ix[:,['Subjid']]).ravel()
fitm = np.array(vtk_fbs.ix[:,['fit_method']]).ravel()
reg = np.array(vtk_fbs.ix[:,['region']]).ravel()
mi_array = np.array([sid, fitm, reg])
index = pd.MultiIndex.from_arrays(mi_array, names=['Subjid', 'fit_method', 'region'])
vtk_fbmi = pd.DataFrame(vtk_fbs.values, index=index, columns=vtk_fbs.columns)

In [6]:
vtk_fbmi.ix[testrange,['fa', 'md', 'FB_reach_dest', 'total_num_FB', 'perc_arr']]

fa           md FB_reach_dest total_num_FB perc_arr
Subjid     fit_method        region                                                                    
sub-bbc101 restore_camino_mf Corpus_Callosum  0.380762  7.87283e-10          2373         2694   88.08%
sub-bbc105 restore_camino_mf Right_IFOF       0.372283  7.72881e-10           205          417   49.16%
sub-bbc106 ols_fsl           Left_IFOF         0.41419   0.00078395            41          592    6.93%
sub-bbc108 ols_dipy          RightPostIntCap  0.470025    0.0007609           134          356   37.64%
sub-bbc113 ols_dipy_mf       LeftPostIntCap   0.472423  0.000766394            75          330   22.73%
sub-bbc116 ols_camino        Corpus_Callosum  0.402936  8.32691e-10          1220         2551   47.82%
           wls_fsl           Right_IFOF       0.386043  0.000625174            44          298   14.77%
sub-bbc118 wls_fsl_mf        Left_IFOF        0.285014  0.000783098            41          298   13.76%
sub-bbc119 wls_dipy_mf       RightPostIntCap  0.410033  0.000917187            90          334   26.95%
sub-bbc120 wls_camino        LeftPostIntCap   0.453479   7.9184e-10            95          357   26.61%
sub-bbc202 wls_camino_mf     Corpus_Callosum  0.401492  7.98441e-10          2603         3237   80.41%
sub-bbc208 NaN               Right_IFOF         0.4536   0.00075223           125          336   37.20%
sub-bbc209 restore_camino    Left_IFOF        0.402138  7.92716e-10             9          514    1.75%
sub-bbc211 ols_fsl           RightPostIntCap  0.419324  0.000686176             4          268    1.49%
sub-bbc231 ols_fsl_mf        LeftPostIntCap   0.403578  0.000750262           191          325   58.77%
sub-bbc241 ols_dipy          Corpus_Callosum  0.416018  0.000812205          1246         2737   45.52%
sub-bbc243 ols_camino        Right_IFOF       0.469592  8.30136e-10           103          326   31.60%
sub-bbc249 ols_camino_mf     Left_IFOF        0.294036  7.90402e-10            16          156   10.26%
           wls_fsl_mf        RightPostIntCap  0.357175   0.00076327           198          256   77.34%
sub-bbc253 wls_dipy          LeftPostIntCap   0.463179  0.000761653           119          350   34.00%

,Subjid,fit_method,region,vtk_file_name,fa,ad,rd,md,vo,length,FB_reach_dest,total_num_FB,%arrived,FB_percent_reach_dest
0,sub-bbc101,restore_camino_mf,Corpus_Callosum,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam_tensor_medfilt_mori_CC.vtk,0.380762,8.562840e-10,7.527851e-10,7.872827e-10,1,220.954910,2373,2694,0.880846,88.08%
1,sub-bbc101,restore_camino_mf,Left_IFOF,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam_tensor_medfilt_mori_Left_IFOF-70.vtk,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000,0,131,0.000000,0.00%
2,sub-bbc101,restore_camino_mf,LeftPostIntCap,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam_tensor_medfilt_mori_LeftPostIntCap-35.vtk,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000,0,308,0.000000,0.00%
3,sub-bbc101,restore_camino_mf,Right_IFOF,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam_tensor_medfilt_mori_Right_IFOF-158.vtk,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000,0,308,0.000000,0.00%
4,sub-bbc101,restore_camino_mf,RightPostIntCap,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam_tensor_medfilt_mori_RightPostIntCap-123.vtk,0.287925,6.078071e-10,7.872144e-10,7.274122e-10,1,196.659332,20,245,0.081633,8.16%
5,sub-bbc101,restore_camino,Corpus_Callosum,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam_tensor_mori_CC.vtk,0.417196,8.699177e-10,7.634652e-10,7.989497e-10,1,199.373734,2046,2707,0.755818,75.58%
6,sub-bbc101,restore_camino,Left_IFOF,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam_tensor_mori_Left_IFOF-70.vtk,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000,0,268,0.000000,0.00%
7,sub-bbc101,restore_camino,LeftPostIntCap,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam_tensor_mori_LeftPostIntCap-35.vtk,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000,0,315,0.000000,0.00%
8,sub-bbc101,restore_camino,Right_IFOF,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam_tensor_mori_Right_IFOF-158.vtk,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000,0,315,0.000000,0.00%
9,sub-bbc101,restore_camino,RightPostIntCap,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam_tensor_mori_RightPostIntCap-123.vtk,0.375120,5.483428e-10,7.894300e-10,7.090674e-10,1,187.962692,8,250,0.032000,3.20%


FB_reach_dest  total_num_FB  %arrived FB_percent_reach_dest
Subjid     fit_method        region                                                                      
sub-bbc101 restore_camino_mf Corpus_Callosum           2373          2694  0.880846                88.08%
                             Left_IFOF                    0           131  0.000000                 0.00%
                             LeftPostIntCap               0           308  0.000000                 0.00%
                             Right_IFOF                   0           308  0.000000                 0.00%
                             RightPostIntCap             20           245  0.081633                 8.16%
           restore_camino    Corpus_Callosum           2046          2707  0.755818                75.58%
                             Left_IFOF                    0           268  0.000000                 0.00%
                             LeftPostIntCap               0           315  0.000000                 0.00%
                             Right_IFOF                   0           315  0.000000                 0.00%
                             RightPostIntCap              8           250  0.032000                 3.20%
           wls_camino_mf     Corpus_Callosum           2414          2742  0.880379                88.04%
                             Left_IFOF                    0           132  0.000000                 0.00%
                             LeftPostIntCap               4           340  0.011765                 1.18%
                             Right_IFOF                   4           340  0.011765                 1.18%
                             RightPostIntCap             29           261  0.111111                11.11%
           wls_camino        Corpus_Callosum           2110          2783  0.758175                75.82%
                             Left_IFOF                    0           273  0.000000                 0.00%
                             LeftPostIntCap               2           333  0.006006                 0.60%
                             Right_IFOF                   2           333  0.006006                 0.60%
                             RightPostIntCap             37           262  0.141221                14.12%
           wls_dipy_mf       Corpus_Callosum           2082          2741  0.759577                75.96%
                             Left_IFOF                    0           265  0.000000                 0.00%
                             LeftPostIntCap               2           331  0.006042                 0.60%
                             Right_IFOF                   2           331  0.006042                 0.60%
                             RightPostIntCap             32           259  0.123552                12.36%
           wls_dipy          Corpus_Callosum           2082          2741  0.759577                75.96%
                             Left_IFOF                    0           265  0.000000                 0.00%
                             LeftPostIntCap               2           331  0.006042                 0.60%
                             Right_IFOF                   2           331  0.006042                 0.60%
                             RightPostIntCap             32           259  0.123552                12.36%
           wls_fsl_mf        Corpus_Callosum           2270          2561  0.886373                88.64%
                             Left_IFOF                    1           122  0.008197                 0.82%
                             LeftPostIntCap               3           288  0.010417                 1.04%
                             Right_IFOF                   3           288  0.010417                 1.04%
                             RightPostIntCap             17           225  0.075556                 7.56%
           wls_fsl           Corpus_Callosum           1950          2563  0.760827                76.08%
                  

In [19]:

vtk_fbs.groupby(level=0).rank('%arrived')

Empty DataFrame
Columns: []
Index: []


In [25]:
vtk_fbs.loc[vtk_fbs['fit_method'].str('ols_fsl_mf'), 'FB_percent_reach_dest']

KeyError: 'fit_method'

,Subjid,fit_method,region,vtk_file_name,FB_reach_dest,total_num_FB,FB_percent_reach_dest
31,sub-bbc105,NaN,RightPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam2fsl_tensor_mori_RightPostIntCap-123.vtk,225,384,41.41%
32,sub-bbc105,wls_dipy_mf,LeftPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_dipy_tensor_medfilt_mori_LeftPostIntCap-35.vtk,203,418,51.44%
33,sub-bbc105,wls_dipy_mf,RightPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_dipy_tensor_medfilt_mori_RightPostIntCap-123.vtk,355,397,10.58%
34,sub-bbc105,wls_dipy,LeftPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_dipy_tensor_mori_LeftPostIntCap-35.vtk,126,408,69.12%
35,sub-bbc105,wls_dipy,RightPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_dipy_tensor_mori_RightPostIntCap-123.vtk,239,385,37.92%
36,sub-bbc105,wls_fsl_mf,LeftPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_medfilt_mori_LeftPostIntCap-35.vtk,203,415,51.08%
37,sub-bbc105,wls_fsl_mf,RightPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_medfilt_mori_RightPostIntCap-123.vtk,349,395,11.65%
38,sub-bbc105,wls_fsl,LeftPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_mori_LeftPostIntCap-35.vtk,118,406,70.94%
39,sub-bbc105,wls_fsl,RightPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_mori_RightPostIntCap-123.vtk,232,378,38.62%
40,sub-bbc106,ols_dipy_mf,LeftPostIntCap,sub-bbc106_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_ols_dipy_tensor_medfilt_mori_LeftPostIntCap-35.vtk,248,458,45.85%


In [18]:
sid_lst = vtk_fbs.Subjid.unique()
fitm_lst = vtk_fbs.fit_method.unique()
region_lst = vtk_fbs.region.unique()
idx_arr = [sid_lst, fitm_lst, region_lst]
tuples = list(zip(sid_lst, fitm_lst, region_lst))

vtk_fbs.set_index(['Subjid', 'fit_method', 'region'], inplace=True)

In [21]:
vtk_fbs['FB_percent_reach_dest']

Subjid      fit_method         region         
sub-bbc101  ols_dipy_mf        LeftPostIntCap     100.00%
                               RightPostIntCap     80.88%
            ols_dipy           LeftPostIntCap     100.00%
                               RightPostIntCap     85.82%
            ols_fsl_mf         LeftPostIntCap      98.96%
                               RightPostIntCap     92.44%
            ols_fsl            LeftPostIntCap      99.34%
                               RightPostIntCap     94.07%
            NaN                LeftPostIntCap     100.00%
                               RightPostIntCap     91.84%
                               LeftPostIntCap     100.00%
                               RightPostIntCap     96.80%
            wls_dipy_mf        LeftPostIntCap      99.11%
                               RightPostIntCap     89.11%
            wls_dipy           LeftPostIntCap      99.40%
                               RightPostIntCap     87.64%
            wls_fsl_mf   

In [3]:
testf = fs / 'bbc' / 'vtk_df_test.csv'

In [5]:
testVTK_DF = pd.read_csv(testf, header=0, index_col=['Subjid', 'fit_method', 'region'])

ValueError: Index Subjid invalid

In [5]:
#this renames col label 'subjectid' with 'vtk_file_name'
testVTK_DF.rename(columns={'subjectid': 'vtk_file_name', 'icountgood': 'FB_reach_dest', 'icounttotal': 'total_num_FB'}, inplace=True)
#makes subj id fm 1st 10 chars of vtk_file_name
testVTK_DF['Subjid'] = testVTK_DF['vtk_file_name'].str[0:10]
#makes fit methods
testVTK_DF.loc[testVTK_DF['vtk_file_name'].str.contains('restore_cam2fsl_tensor_medf'), 'fit_method'] = 'restore_camino'
testVTK_DF.loc[testVTK_DF['vtk_file_name'].str.contains('wls_fsl_tensor_medf'), 'fit_method'] = 'wls_fsl_mf'
testVTK_DF.loc[testVTK_DF['vtk_file_name'].str.contains('ols_dipy_tensor_med'), 'fit_method'] = 'ols_dipy_mf'
testVTK_DF.loc[testVTK_DF['vtk_file_name'].str.contains('wls_cam_tensor_medf'), 'fit_method'] = 'wls_camino_mf'
testVTK_DF.loc[testVTK_DF['vtk_file_name'].str.contains('wls_cam_tensor_mori'), 'fit_method'] = 'wls_camino'
#makes region names for regions col.
testVTK_DF.loc[testVTK_DF['vtk_file_name'].str.contains('LeftPostIntCap'), 'region'] = 'LeftPostIntCap'
testVTK_DF.loc[testVTK_DF['vtk_file_name'].str.contains('RightPostIntCap'), 'region'] = 'RightPostIntCap'
testVTK_DF.loc[testVTK_DF['vtk_file_name'].str.contains('mori_CC'), 'region'] = 'Corpus_Callosum'



In [9]:
testVTK_DF.ix[2:4,['Subjid', 'fit_method', 'region', 'vtk_file_name', 'FB_reach_dest', 'total_num_FB']]
#testVTK_DF.set_index(['Subjid', 'fit_method', 'region'], inplace=True)
testVTK_DF['%arrived'] = 1 - testVTK_DF['total_num_FB'].sub(testVTK_DF['FB_reach_dest']).div(testVTK_DF['total_num_FB'])
testVTK_DF.ix[:,['%arrived', 'fa']]

%arrived        fa
Subjid     fit_method     region                             
sub-bbc101 ols_dipy_mf    LeftPostIntCap   0.000000  0.000000
           restore_camino RightPostIntCap  0.081633  0.287925
sub-bbc105 restore_camino RightPostIntCap  0.881612  0.357323
           wls_fsl_mf     LeftPostIntCap   0.489157  0.372089
sub-bbc116 wls_camino_mf  Corpus_Callosum  0.660230  0.352310
sub-bbc106 wls_camino     Corpus_Callosum  0.675721  0.467040
sub-bbc116 wls_fsl_mf     LeftPostIntCap   0.415335  0.305885
                          RightPostIntCap  0.148515  0.322960
sub-bbc118 restore_camino RightPostIntCap  0.428571  0.370943

In [ ]:
testVTK_DF.groupby(level='Subjid').

In [14]:
import statsmodels
statsmodels.__version__

'0.6.1'

In [12]:
testVTK_DF.set_index(['Subjid', 'fit_method', 'region'], inplace=True)
testVTK_DF.columns

KeyError: 'Subjid'

In [15]:
testVTK_DF.reset_index
#testVTK_DF.set_index(['Subjid', 'fit_method', 'region'], inplace=True)
testVTK_DF.columns

Index([u'vtk_file_name', u'fa', u'ad', u'rd', u'md', u'vo', u'length', u'FB_reach_dest', u'total_num_FB'], dtype='object')

In [10]:
testVTK_DF.index.get_level_values('region')

Index([u'LeftPostIntCap', u'RightPostIntCap', u'RightPostIntCap', u'LeftPostIntCap', u'Corpus_Callosum', u'Corpus_Callosum', u'LeftPostIntCap', u'RightPostIntCap', u'RightPostIntCap'], dtype='object', name=u'region')

In [21]:
testVTK_DF.loc[testVTK_DF['fit_method'].str('ols_fsl_mf'), 'FB_percent_reach_dest']

KeyError: 'fit_method'

In [18]:
testVTK_DF.xs('Subjid', level=0)

KeyError: 'Subjid'